<a href="https://colab.research.google.com/github/boseull/pytorch/blob/main/pytorch_%EB%B9%A0%EB%A5%B8%EC%8B%9C%EC%9E%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

dataset : 샘플과 라벨 저장  
dataloader : dataset을 iterable로 감쌈

## 데이터 작업

In [2]:
training_data = datasets.FashionMNIST(root="data", train=True, download=True, transform=ToTensor(),)

test_data = datasets.FashionMNIST(root="data", train=False, download=True, transform=ToTensor(),)

batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print("Shape of X [N, C, H, W]: ", X.shape)
  print("Shape of y: ", y.shape, y.dtype)
  break

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## 모델 만들기

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cpu device


In [4]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(nn.Linear(28*28, 512), nn.ReLU(), nn.Linear(512, 512), nn.ReLU(), nn.Linear(512,10), nn.ReLU())

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


## 모델 매개변수 최적화하기

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    pred = model(X)
    loss = loss_fn(pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>f}  [{current:>5d}/{size:>5d}]")

In [6]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [7]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n-----------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)

print("Done!")

Epoch 1
-----------------------------
loss: 2.302063  [    0/60000]
loss: 2.293818  [ 6400/60000]
loss: 2.282967  [12800/60000]
loss: 2.282629  [19200/60000]
loss: 2.271035  [25600/60000]
loss: 2.256742  [32000/60000]
loss: 2.258848  [38400/60000]
loss: 2.240636  [44800/60000]
loss: 2.236256  [51200/60000]
loss: 2.209892  [57600/60000]
Test Error: 
 Accuracy: 42.7%, Avg loss: 2.220421

Epoch 2
-----------------------------
loss: 2.239327  [    0/60000]
loss: 2.221404  [ 6400/60000]
loss: 2.181323  [12800/60000]
loss: 2.180812  [19200/60000]
loss: 2.173100  [25600/60000]
loss: 2.144914  [32000/60000]
loss: 2.163766  [38400/60000]
loss: 2.129486  [44800/60000]
loss: 2.131375  [51200/60000]
loss: 2.069392  [57600/60000]
Test Error: 
 Accuracy: 44.0%, Avg loss: 2.094501

Epoch 3
-----------------------------
loss: 2.145617  [    0/60000]
loss: 2.103686  [ 6400/60000]
loss: 2.020697  [12800/60000]
loss: 2.018820  [19200/60000]
loss: 2.021752  [25600/60000]
loss: 1.975387  [32000/60000]
loss

##save model

In [8]:
torch.save(model.state_dict(), "model.pth")
print('Saved PyTorch Model State to model.pth')

model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

classes = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot",]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f'predicted: "{predicted}", Actual: "{actual}"')

Saved PyTorch Model State to model.pth
predicted: "Ankle boot", Actual: "Ankle boot"
